In [47]:
from z3 import Optimize, Real, If, sat
from fractions import Fraction

def compute_corporate_income_tax_optimize(
    OperatingRevenueTotal=0, SalesReturn=0, SalesAllowance=0,
    OperatingCost=0, OperatingExpensesLosses=0,
    NonOperatingRevenueTotal=0, NonOperatingLossExpenses=0,
    Prev10LossDeduction=0, TaxIncentiveExempt=0,
    ExemptSecuritiesIncome=0, ExemptLandIncome=0,
    Article4_4HouseLandGain=0,
    is_full_year=True,    # 營業期間是否滿一年
    m_partial=12,         # 營業期間未滿一年時，營業月份數
    free_vars: list = None  # 指定哪些變數為自由變數（僅做範圍檢查，不直接固定）
):
    if free_vars is None:
        free_vars = []

    # 使用 Optimize 求解器
    opt = Optimize()

    # --- 1. 定義 12 個變數 (名稱後綴 _z) ---
    OperatingRevenueTotal_z    = Real('OperatingRevenueTotal_z')
    SalesReturn_z              = Real('SalesReturn_z')
    SalesAllowance_z           = Real('SalesAllowance_z')
    OperatingCost_z            = Real('OperatingCost_z')
    OperatingExpensesLosses_z  = Real('OperatingExpensesLosses_z')
    NonOperatingRevenueTotal_z = Real('NonOperatingRevenueTotal_z')
    NonOperatingLossExpenses_z = Real('NonOperatingLossExpenses_z')
    Prev10LossDeduction_z      = Real('Prev10LossDeduction_z')
    TaxIncentiveExempt_z       = Real('TaxIncentiveExempt_z')
    ExemptSecuritiesIncome_z   = Real('ExemptSecuritiesIncome_z')
    ExemptLandIncome_z         = Real('ExemptLandIncome_z')
    Article4_4HouseLandGain_z  = Real('Article4_4HouseLandGain_z')

    # 建立「輸入值 -> Z3 變數」字典，每個 tuple 包含：(z3變數, 原始值, 約束條件列表)
    params = {
        "OperatingRevenueTotal":    (OperatingRevenueTotal_z,    OperatingRevenueTotal,    [lambda var: var >= 0]),
        "SalesReturn":              (SalesReturn_z,              SalesReturn,              [lambda var: var >= 0]),
        "SalesAllowance":           (SalesAllowance_z,           SalesAllowance,           [lambda var: var >= 0]),
        "OperatingCost":            (OperatingCost_z,            OperatingCost,            [lambda var: var >= 0]),
        "OperatingExpensesLosses":  (OperatingExpensesLosses_z,  OperatingExpensesLosses,  [lambda var: var >= 0]),
        "NonOperatingRevenueTotal": (NonOperatingRevenueTotal_z, NonOperatingRevenueTotal, [lambda var: var >= 0]),
        "NonOperatingLossExpenses": (NonOperatingLossExpenses_z, NonOperatingLossExpenses, [lambda var: var >= 0]),
        "Prev10LossDeduction":      (Prev10LossDeduction_z,      Prev10LossDeduction,      [lambda var: var >= 0]),
        "TaxIncentiveExempt":       (TaxIncentiveExempt_z,       TaxIncentiveExempt,       [lambda var: var >= 0]),
        "ExemptSecuritiesIncome":   (ExemptSecuritiesIncome_z,   ExemptSecuritiesIncome,   [lambda var: var >= 0]),
        "ExemptLandIncome":         (ExemptLandIncome_z,         ExemptLandIncome,         [lambda var: var >= 0]),
        "Article4_4HouseLandGain":  (Article4_4HouseLandGain_z,  Article4_4HouseLandGain,  [lambda var: var >= 0])
    }

    # 為每個參數加入約束：若不在 free_vars 中則固定其值；在 free_vars 中僅保留額外約束（例如範圍檢查）
    for param_name, (z3_var, value, constraints) in params.items():
        if free_vars and param_name in free_vars:
            for cons in constraints:
                opt.add(cons(z3_var))
        else:
            opt.add(z3_var == value)
            for cons in constraints:
                opt.add(cons(z3_var))

    # --- 2. 中間計算變數及稅額計算 ---
    OperatingRevenueNet_z   = Real('OperatingRevenueNet_z')
    OperatingGrossProfit_z  = Real('OperatingGrossProfit_z')
    OperatingNetProfit_z    = Real('OperatingNetProfit_z')
    YearlyIncome_z          = Real('YearlyIncome_z')
    T_z                     = Real('T_z')  # 稅額

    opt.add(OperatingRevenueNet_z == OperatingRevenueTotal_z - SalesReturn_z - SalesAllowance_z)
    opt.add(OperatingGrossProfit_z == OperatingRevenueNet_z - OperatingCost_z)
    opt.add(OperatingNetProfit_z == OperatingGrossProfit_z - OperatingExpensesLosses_z)
    opt.add(YearlyIncome_z == OperatingNetProfit_z + NonOperatingRevenueTotal_z - NonOperatingLossExpenses_z)

    P_z = Real('P_z')
    opt.add(P_z == YearlyIncome_z 
           - Prev10LossDeduction_z 
           - TaxIncentiveExempt_z 
           - ExemptSecuritiesIncome_z 
           - ExemptLandIncome_z 
           - Article4_4HouseLandGain_z)

    # 根據是否滿一整年，建立分段計算公式
    if is_full_year:
        T_expr = If(P_z <= 120000,
                    0,
                    If(P_z <= 200000,
                       (P_z - 120000) * 0.5,
                       P_z * 0.20))
    else:
        P_adj_z = Real('P_adj_z')
        opt.add(P_adj_z == P_z * 12 / m_partial)
        T_expr = If(P_adj_z <= 120000,
                    0,
                    If(P_adj_z <= 200000,
                       (P_adj_z - 120000) * 0.5 * (m_partial / 12),
                       P_adj_z * 0.20 * (m_partial / 12)))
    piecewise_T_z = Real('piecewise_T_z')
    opt.add(piecewise_T_z == T_expr)
    opt.add(T_z == piecewise_T_z)

    # --- 3. 利用 push/pop 固定 free_vars 以獲得初始稅額（基準解） ---
    if free_vars:
        opt.push()
        for param in free_vars:
            z3_var, orig_value, _ = params[param]
            opt.add(z3_var == orig_value)
        if opt.check() == sat:
            fixed_model = opt.model()
            fixed_tax = fixed_model[T_z].as_long()
        else:
            fixed_tax = None
        opt.pop()
    else:
        if opt.check() == sat:
            fixed_tax = opt.model()[T_z].as_long()
        else:
            fixed_tax = None

    print("初始計算（固定 free_vars）稅額：{}".format(fixed_tax))
    print("--------------------------------------------------\n")
    print("自由變數有：{}\n".format(free_vars))

    # --- 4. 進入最佳化迭代，每次印出變數調整與稅額變化 ---
    print("=== 優化過程開始 ===\n")
    iteration = 0
    best_tax = fixed_tax
    optimal_model = None

    # 利用 while 迴圈模擬每次最佳化，並逐步加上約束 T_z < best_tax
    while True:
        if opt.check() != sat:
            print("第 {} 次: unsat，無解，已是最佳解".format(iteration + 1))
            break
        m = opt.model()
        current_tax = m[T_z].as_long()
        # 若是第一次或找到更低的稅額就印出該次資訊
        if iteration == 0 or current_tax < best_tax:
            print("----- 第 {} 次優化 -----".format(iteration + 1))
            print("當前稅額: {}".format(current_tax))
            iter_differences = {}
            for param_name, (z3_var, orig_value, _) in params.items():
                try:
                    opt_value = m.eval(z3_var).as_long()
                except Exception:
                    opt_value = None
                # 只列印變數有調整的項目
                if opt_value is not None and opt_value != orig_value:
                    iter_differences[param_name] = {
                        "original": orig_value,
                        "optimized": opt_value,
                        "difference": opt_value - orig_value
                    }
            if iter_differences:
                print("變更的變數:")
                for key, diff in iter_differences.items():
                    print("  {}: 原始 = {}, 優化後 = {}, 差異 = {}".format(key, diff["original"], diff["optimized"], diff["difference"]))
            else:
                print("無任何變數變動")
            print("-------------------------")
            best_tax = current_tax
            optimal_model = m
        else:
            print("----- 第 {} 次優化: 無變化，已是最佳解 -----".format(iteration + 1))
            break
        # 為逼迫下一次更低稅額，加上新的約束
        opt.add(T_z < best_tax)
        iteration += 1

    print("\n=== 優化結束，共 {} 次迭代，最終稅額 = {} ===\n".format(iteration, best_tax))

    # --- 5. 建立最終結果參數字典與變數差異訊息 ---
    final_params = {}
    differences = {}
    # 若最佳模型尚未獲得，取目前模型
    if optimal_model is None:
        optimal_model = opt.model()
    for param_name, (z3_var, orig_value, _) in params.items():
        try:
            opt_value = optimal_model.eval(z3_var).as_long()
        except Exception:
            opt_value = None
        final_params[param_name] = {
            "value": opt_value,
            "type": "free" if free_vars and param_name in free_vars else "fixed"
        }
        if opt_value is not None and opt_value != orig_value:
            differences[param_name] = {
                "original": orig_value,
                "optimized": opt_value,
                "difference": opt_value - orig_value
            }

    # 回傳最佳稅額、最終參數狀態以及變數變動訊息
    return best_tax, final_params, differences

# 範例呼叫
if __name__ == "__main__":
    # 假設我們希望讓 "NonOperatingRevenueTotal" 為自由變數，讓其在優化時自由調整
    best, final_params, diffs = compute_corporate_income_tax_optimize(
        OperatingRevenueTotal=1000000, SalesReturn=50000, SalesAllowance=10000,
        OperatingCost=300000, OperatingExpensesLosses=50000,
        NonOperatingRevenueTotal=20000, NonOperatingLossExpenses=15000,
        Prev10LossDeduction=0, TaxIncentiveExempt=0,
        ExemptSecuritiesIncome=0, ExemptLandIncome=0,
        Article4_4HouseLandGain=0,
        is_full_year=True,
        m_partial=12,
        free_vars=["NonOperatingRevenueTotal"]
    )

    print("\n最終結果:")
    print("最佳稅額 = {}".format(best))
    print("最終參數狀態 = {}".format(final_params))
    print("變數變動訊息 = {}".format(diffs))


初始計算（固定 free_vars）稅額：119000
--------------------------------------------------

自由變數有：['NonOperatingRevenueTotal']

=== 優化過程開始 ===

----- 第 1 次優化 -----
當前稅額: 115000
變更的變數:
  NonOperatingRevenueTotal: 原始 = 20000, 優化後 = 0, 差異 = -20000
-------------------------
第 2 次: unsat，無解，已是最佳解

=== 優化結束，共 1 次迭代，最終稅額 = 115000 ===


最終結果:
最佳稅額 = 115000
最終參數狀態 = {'OperatingRevenueTotal': {'value': 1000000, 'type': 'fixed'}, 'SalesReturn': {'value': 50000, 'type': 'fixed'}, 'SalesAllowance': {'value': 10000, 'type': 'fixed'}, 'OperatingCost': {'value': 300000, 'type': 'fixed'}, 'OperatingExpensesLosses': {'value': 50000, 'type': 'fixed'}, 'NonOperatingRevenueTotal': {'value': 0, 'type': 'free'}, 'NonOperatingLossExpenses': {'value': 15000, 'type': 'fixed'}, 'Prev10LossDeduction': {'value': 0, 'type': 'fixed'}, 'TaxIncentiveExempt': {'value': 0, 'type': 'fixed'}, 'ExemptSecuritiesIncome': {'value': 0, 'type': 'fixed'}, 'ExemptLandIncome': {'value': 0, 'type': 'fixed'}, 'Article4_4HouseLandGain': {'value'